In [1]:
import xgboost as xgb
print(xgb.__version__)

0.90


In [2]:
from xgboost import plot_importance
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_excel('./excel_data/advanced5_train_data.xlsx')
test = pd.read_excel('./excel_data/advanced5_test_data.xlsx')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11509 entries, 0 to 11508
Data columns (total 59 columns):
item_key                   11509 non-null int64
body_length                11509 non-null float64
body_width                 11509 non-null float64
body_height                11509 non-null float64
wavyfin_length             11509 non-null float64
wavyfin_width              11509 non-null float64
wavyfin_height             11509 non-null float64
op10_length                11509 non-null float64
op10_width                 11509 non-null float64
op10_height                11509 non-null float64
op10_electricity_sensor    11509 non-null float64
op10_process_time          11509 non-null int64
op10_test                  11509 non-null int64
op10_time_stamp            11509 non-null int64
pipe1_length               11509 non-null float64
pipe1_width                11509 non-null float64
pipe1_height               11509 non-null float64
op20_length                11509 non-null float64

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 59 columns):
item_key                   2879 non-null int64
body_length                2879 non-null float64
body_width                 2879 non-null float64
body_height                2879 non-null float64
wavyfin_length             2879 non-null float64
wavyfin_width              2879 non-null float64
wavyfin_height             2879 non-null float64
op10_length                2879 non-null float64
op10_width                 2879 non-null float64
op10_height                2879 non-null float64
op10_electricity_sensor    2879 non-null float64
op10_process_time          2879 non-null int64
op10_test                  2879 non-null int64
op10_time_stamp            2879 non-null int64
pipe1_length               2879 non-null float64
pipe1_width                2879 non-null float64
pipe1_height               2879 non-null float64
op20_length                2879 non-null float64
op20_width         

In [29]:
X_train = train.drop(['op60_time_stamp', 'op60_process_time', 
                   'op60_height', 'op60_width', 'op60_length', 'op60_test', 
                   'op50_time_stamp', 'op50_process_time', 
                   'op50_height', 'op50_width', 'op50_length', 'op50_temperature',
                   'op40_time_stamp', 'op40_process_time', 
                   'op40_height', 'op40_width', 'op40_length', 'op40_temperature', 'op40_test',
                   'item_key'], axis = 1)
X_train.head()

,body_length,body_width,body_height,wavyfin_length,wavyfin_width,wavyfin_height,op10_length,op10_width,op10_height,op10_electricity_sensor,...,op30_electricity_sensor,op30_process_time,op30_test,op30_time_stamp,flange1_length,flange1_width,flange1_height,flange2_length,flange2_width,flange2_height
0,200.00291,100.00127,50.00120,100.00060,49.99947,59.99918,200.00159,99.99886,60.00084,96.69760,...,95.42282,10,1,70,29.99403,79.99724,39.99997,29.99989,80.00016,40.00316
1,200.00058,100.00023,49.99356,99.99844,50.00081,59.99694,200.00291,99.99886,59.99918,93.02913,...,92.34298,10,0,80,29.99700,79.99948,39.99735,30.00253,80.00158,39.99826
2,199.99809,99.99980,49.99937,99.99760,49.99630,60.00232,200.00058,99.99886,59.99694,90.62536,...,91.82355,10,0,90,30.00493,79.99883,39.99943,29.99823,79.99929,40.00145
3,199.99733,99.99884,50.00222,100.00120,49.99800,59.99907,199.99809,99.99886,60.00232,99.30119,...,93.45783,10,0,100,30.00082,80.00320,40.00309,30.00022,79.99437,39.99721
4,200.00176,99.99959,50.00200,100.00269,50.00212,59.99901,199.99733,99.99886,59.99907,96.10423,...,97.94294,10,0,110,30.00100,80.00036,40.00182,29.99810,80.00011,39.99834


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11509 entries, 0 to 11508
Data columns (total 33 columns):
body_length                11509 non-null float64
body_width                 11509 non-null float64
body_height                11509 non-null float64
wavyfin_length             11509 non-null float64
wavyfin_width              11509 non-null float64
wavyfin_height             11509 non-null float64
op10_length                11509 non-null float64
op10_width                 11509 non-null float64
op10_height                11509 non-null float64
op10_electricity_sensor    11509 non-null float64
op10_test                  11509 non-null int64
pipe1_length               11509 non-null float64
pipe1_width                11509 non-null float64
pipe1_height               11509 non-null float64
op20_length                11509 non-null float64
op20_width                 11509 non-null float64
op20_height                11509 non-null float64
op20_electricity_sensor    11509 non-null f

In [8]:
Y_train = train['op60_test']
Y_train.head()

0    0
1    1
2    1
3    0
4    0
Name: op60_test, dtype: int64

In [25]:
X_test = test.drop(['op60_time_stamp', 'op60_process_time', 
                   'op60_height', 'op60_width', 'op60_length', 'op60_test', 
                   'op50_time_stamp', 'op50_process_time', 
                   'op50_height', 'op50_width', 'op50_length', 'op50_temperature',
                   'op40_time_stamp', 'op40_process_time', 
                   'op40_height', 'op40_width', 'op40_length', 'op40_temperature', 'op40_test',
                   'item_key'], axis = 1)
X_test.head()

,body_length,body_width,body_height,wavyfin_length,wavyfin_width,wavyfin_height,op10_length,op10_width,op10_height,op10_electricity_sensor,...,op30_electricity_sensor,op30_process_time,op30_test,op30_time_stamp,flange1_length,flange1_width,flange1_height,flange2_length,flange2_width,flange2_height
0,199.99658,100.00182,49.99818,100.00554,49.99813,60.00126,200.00064,99.99886,59.99634,99.22668,...,97.02756,10,0,115160,30.00049,80.00351,39.99782,29.99985,80.00118,39.99765
1,199.99713,99.99860,50.00096,100.00927,50.00083,60.00034,199.99658,99.99886,60.00126,91.08281,...,93.87609,10,0,115170,30.00138,80.00063,40.00158,30.00103,79.99501,40.00227
2,200.00282,100.00529,50.00384,100.00144,49.99816,59.99651,199.99713,99.99886,60.00034,98.64852,...,96.04069,10,0,115180,29.99943,80.00266,40.00002,29.99942,79.99698,40.00234
3,200.00169,100.00001,50.00289,99.99753,49.99951,59.99916,200.00282,99.99886,59.99651,92.95297,...,99.03526,10,0,115190,30.00059,80.00214,40.00159,29.99514,79.99716,40.00065
4,200.00132,100.00324,50.00222,99.99392,50.00096,60.00247,200.00169,99.99886,59.99916,96.80218,...,94.46605,10,0,115200,30.00098,80.00104,39.99760,30.00693,80.00156,39.99734


In [26]:
Y_test = test['op60_test']
Y_test.head()

0    0
1    0
2    0
3    0
4    0
Name: op60_test, dtype: int64

In [27]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2879 entries, 0 to 2878
Data columns (total 39 columns):
body_length                2879 non-null float64
body_width                 2879 non-null float64
body_height                2879 non-null float64
wavyfin_length             2879 non-null float64
wavyfin_width              2879 non-null float64
wavyfin_height             2879 non-null float64
op10_length                2879 non-null float64
op10_width                 2879 non-null float64
op10_height                2879 non-null float64
op10_electricity_sensor    2879 non-null float64
op10_process_time          2879 non-null int64
op10_test                  2879 non-null int64
op10_time_stamp            2879 non-null int64
pipe1_length               2879 non-null float64
pipe1_width                2879 non-null float64
pipe1_height               2879 non-null float64
op20_length                2879 non-null float64
op20_width                 2879 non-null float64
op20_height      

In [28]:
X_train.shape

(11509, 33)

In [13]:
print(train['op60_test'].value_counts())

0    11142
1      367
Name: op60_test, dtype: int64


In [14]:
print(test['op60_test'].value_counts())

0    2789
1      90
Name: op60_test, dtype: int64


In [31]:
dtrain = xgb.DMatrix(data = X_train, label = Y_train)
dtest = xgb.DMatrix(data = X_test, label = Y_test)

In [32]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]

In [68]:
params = {'max_depth' : 10,
         'eta': 0.001,
         'objective' : 'binary:logistic',
          'eval_metric' : 'error',
         'early_stopping':200
         }
num_rounds = 1000

In [69]:
xgb_model = xgb.train(params = params, dtrain = dtrain, num_boost_round = num_rounds,
                     early_stopping_rounds = 200, evals = wlist)

[0]	train-error:0.031888	eval-error:0.031261
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 200 rounds.
[1]	train-error:0.031888	eval-error:0.031261
[2]	train-error:0.031888	eval-error:0.031261
[3]	train-error:0.031888	eval-error:0.031261
[4]	train-error:0.031888	eval-error:0.031261
[5]	train-error:0.031888	eval-error:0.031261
[6]	train-error:0.031888	eval-error:0.031261
[7]	train-error:0.031888	eval-error:0.031261
[8]	train-error:0.031888	eval-error:0.031261
[9]	train-error:0.031888	eval-error:0.031261
[10]	train-error:0.031888	eval-error:0.031261
[11]	train-error:0.031888	eval-error:0.031261
[12]	train-error:0.031888	eval-error:0.031261
[13]	train-error:0.031888	eval-error:0.031261
[14]	train-error:0.031888	eval-error:0.031261
[15]	train-error:0.031888	eval-error:0.031261
[16]	train-error:0.031888	eval-error:0.031261
[17]	train-error:0.031888	eval-error:0.031261
[18]	train-error:0.031888	eval-error

[174]	train-error:0.031888	eval-error:0.031261
[175]	train-error:0.031888	eval-error:0.031261
[176]	train-error:0.031888	eval-error:0.031261
[177]	train-error:0.031888	eval-error:0.031261
[178]	train-error:0.031888	eval-error:0.031261
[179]	train-error:0.031888	eval-error:0.031261
[180]	train-error:0.031888	eval-error:0.031261
[181]	train-error:0.031888	eval-error:0.031261
[182]	train-error:0.031888	eval-error:0.031261
[183]	train-error:0.031888	eval-error:0.031261
[184]	train-error:0.031888	eval-error:0.031261
[185]	train-error:0.031888	eval-error:0.031261
[186]	train-error:0.031888	eval-error:0.031261
[187]	train-error:0.031888	eval-error:0.031261
[188]	train-error:0.031888	eval-error:0.031261
[189]	train-error:0.031888	eval-error:0.031261
[190]	train-error:0.031888	eval-error:0.031261
[191]	train-error:0.031888	eval-error:0.031261
[192]	train-error:0.031888	eval-error:0.031261
[193]	train-error:0.031888	eval-error:0.031261
[194]	train-error:0.031888	eval-error:0.031261
[195]	train-e

In [70]:
pred_probs = xgb_model.predict(dtest)
print(np.round(pred_probs[:10], 3))

print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨 ')
preds = [1 if x > 0.5 else 0 for x in pred_probs]
print('예측값 100개만 표시: ', preds[:100])

[0.415 0.434 0.415 0.415 0.415 0.415 0.415 0.415 0.415 0.415]
predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨 
예측값 100개만 표시:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [71]:
from sklearn.metrics import accuracy_score, precision_score , recall_score , confusion_matrix

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}'.format(accuracy , precision ,recall))

In [72]:
get_clf_eval(Y_test, preds)

오차 행렬
[[2789    0]
 [  90    0]]
정확도: 0.9687, 정밀도: 0.0000, 재현율: 0.0000
